# Predicting the result of 2021 Belgian GP

### Since I have only P2 times, I will use those to make predictions

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler


# Read the data

In [ ]:

data = pd.read_csv("../input/spa2021/final_dummies.csv")

data = data.drop('Unnamed: 0', axis =1)

In [ ]:
data.shape

## Consider the results of past 3 seasons and all the belgian GP from 1983 onwards.

In [ ]:
ls = data[data.season>2018]
ls = ls[data.circuit_id_spa != 1]

sp = data[data.circuit_id_spa ==1]

train = pd.concat([ls, sp], join = 'inner')

## Preparing the data

In [ ]:
X_train = train.drop(['driver', 'podium', 'round', 'season'], axis = 1)
y_train = train.podium

scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)

### Fitting a classification model using auto-keras

In [ ]:
!pip install autokeras

In [ ]:
import tensorflow as tf
import autokeras as ak

clf = ak.StructuredDataClassifier(overwrite=True, max_trials=3)
clf.fit(X_train, y_train, epochs=350)

## Making Predictions

In [ ]:

X_test = pd.read_csv('../input/spa2021/test_template.csv')
drivers = X_test.driver
X_test = X_test.drop(['driver', 'podium', 'round', 'season'], axis = 1)
X_test = pd.DataFrame(scaler.transform(X_test), columns = X_test.columns)

z = clf.predict(X_test)

res = pd.DataFrame({'Driver': drivers})
res['pos'] = z.astype(int)


In [ ]:
res

### Looks like Max is set to win with Bottas and Hamiltion bringing up the podium.

### Stroll, Gasly, Norris are expected to perform well.

##### While Gasly and Norris are expected here, stroll probably comes from the fact that racing point/ force india/ aston martin have traditionally performed very well in Spa and Stroll is pretty good in rain (Monza 2017, Germany 2019, Turkey 2020).

### Ferraris are expected to struggle as per the model.

### A shocking outlier is Perez

### Giovinazzi got a 25 as he DSQ'ed from previous Belgian GP's and lack of data hurts him.

### Same can be said for tsunoda, who's unusually high position can be attributed to gasly's strong performances in a similar car

# However these results are based on P2 results and Final predictions will depend on Qualifying performance and grid start order and weather conditions.

In [ ]:
model = clf.export_model()

In [ ]:
model.save('model_spa_2021')

# Predictions after Qualification

In [ ]:
X_test = pd.read_csv('../input/spa2021/test_template - Copy.csv')
drivers = X_test.driver
X_test = X_test.drop(['driver', 'podium', 'round', 'season'], axis = 1)
X_test = pd.DataFrame(scaler.transform(X_test), columns = X_test.columns)

z = clf.predict(X_test)

res = pd.DataFrame({'Driver': drivers})
res['pos'] = z.astype(int)


In [ ]:
res

## A note about predictions

Wet weather races are very chaotic and difficult to predict, hence these predictions can be very wrong. Also tracks like spa, baku etc. with long lap times and high safety car chances give a very difficult time to models.